# MLP for POS Tagging

In [1]:
import pyconll, keras, pickle, os, random, nltk, datetime, warnings, gc, urllib.request, zipfile
import numpy as np
import math
from scipy.sparse import hstack, vstack
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
from numpy.random import seed
from sklearn.model_selection import train_test_split

# automatic module reloading
%load_ext autoreload
%autoreload 2

# For reproducibility
np.random.seed(3) 
random.seed(3)

2023-03-11 15:31:32.935856: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# add parent directory to path for imports to work
import sys
sys.path.append('..')

# src imports
from src.utils import get_root_dir, untag
from src.parser import format_data, embeddings_init
from src.data_helpers import vectorize, preprocess_unlabelled_test_data

In [3]:
# functions 
def compare_with_test_set(predicted_data, correct_set):
    total = 0
    correct = 0
    for predicted_sentence, correct_sentence in zip(predicted_data, correct_set):
        for predicted_word, correct_word in zip(predicted_sentence, correct_sentence):
            total = total + 1
            if predicted_word[1] == correct_word[1]:
                #print(predicted_word[0], predicted_word[1], correct_word[1])
                correct = correct + 1
    
    accuracy = (correct / total) * 100
    return accuracy

In [4]:
ROOT_DIR = get_root_dir() 
POS_DIR = os.path.join(ROOT_DIR, 'dataset')
pos_train = os.path.join(POS_DIR, "train.txt")

In [5]:
train_sentences = format_data(pos_train)

In [6]:
print("Tagged sentences in train set: ", len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))

Tagged sentences in train set:  8936
Tagged words in train set: 211727


In [6]:
# # Dataset Split
# total_train = len(train_sentences)
# print(total_train)
# val_split = math.floor(0.2 * total_train)
# print(val_split)
# train_sentences, val_sentences = train_sentences[:(total_train-val_split)], train_sentences[(total_train-val_split):]
# print(len(train_sentences), len(val_sentences))
# print(len(train_sentences)+len(val_sentences))

In [7]:
# print(train_sentences[0])

### Embeddings

In [9]:
# Initialize Embeddings
embeddings_path = os.path.join(get_root_dir(), 'dataset')
t_ini = datetime.datetime.now()
print('Initializing embeddings')
embeddings = embeddings_init(str(embeddings_path))
print('Initialiation completed')
t_fin = datetime.datetime.now()
print('Embeddings loaded in {} seconds'.format((t_fin - t_ini).total_seconds()))

Initializing embeddings
Initialiation completed
Embeddings loaded in 250.935922 seconds


In [10]:
w2c = dict()
for item in embeddings.key_to_index:
    w2c[item] = embeddings.key_to_index[item]

In [11]:
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)
np.random.seed(3)
oov = np.random.uniform(-0.25, 0.25, dim)

In [12]:
X_train, y_train = vectorize(embeddings, oov, train_sentences, window=1)

Embeddings window method
Vectorizing Dataset...
Vectorizing train...
Dataset vectorized.
Train shape: (211727, 900)


In [14]:
X_train.shape

(211727, 900)

In [15]:
classes = sorted(list(set(y_train)))
print(classes)

['#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [42]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [16]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
# y_val = le.transform(y_val)
y_train = keras.utils.to_categorical(y_train)
# y_val = keras.utils.to_categorical(y_val)

print(y_train.shape)

(211727, 44)


In [35]:
# model = Sequential()
# # In the first layer, we specify the input data shape

# model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(y_train.shape[1], activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [36]:
# nb_epoch = 10
# batch_size = 128
# early_stopping = EarlyStopping(monitor = 'val_acc', patience = 5)
# history = model.fit(X_train, y_train,
#                     epochs=nb_epoch,
#                     batch_size=batch_size,
#                     shuffle=True,
#                     validation_data=(X_val, y_val),
#                     verbose=1,
#                     callbacks=[early_stopping])

In [37]:
# X_full_train = vstack((X_train, X_val)).tocsr()
# y_full_train = np.append(y_train, y_val, axis=0)

In [38]:
# X_train.shape

(169381, 70531)

In [20]:
def get_mlp_model(input_dim):
    """
    Define a MLP model
    """
    model = Sequential()

    model.add(Dense(512, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [21]:
model = get_mlp_model(X_train.shape[1])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               461312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 44)                2860      
Total params: 497,004
Trainable params: 497,004
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    verbose=1,
                   ) 

2023-03-11 14:46:07.610300: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 762217200 exceeds 10% of free system memory.
2023-03-11 14:46:08.757465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-11 14:46:08.791268: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1700000000 Hz


Epoch 1/20


2023-03-11 14:46:10.153685: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


6617/6617 [==============================] - 35s 4ms/step - loss: 0.7402 - accuracy: 0.7988
Epoch 2/20
6617/6617 [==============================] - 26s 4ms/step - loss: 0.2310 - accuracy: 0.9305
Epoch 3/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1890 - accuracy: 0.9426
Epoch 4/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1695 - accuracy: 0.9473
Epoch 5/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1593 - accuracy: 0.9503
Epoch 6/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1480 - accuracy: 0.9530
Epoch 7/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1421 - accuracy: 0.9550
Epoch 8/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1368 - accuracy: 0.9576
Epoch 9/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1290 - accuracy: 0.9602
Epoch 10/20
6617/6617 [==============================] - 27s 4ms/step - loss: 0.1248

In [142]:
# model.save("my_model.h5")

In [45]:
# model = keras.models.load_model(save_format='h5', "my_model.h5")

2023-03-10 21:59:31.257721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 21:59:31.257913: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 21:59:31.260761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-10 21:59:31.261050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-10 21:59:31.261091: W tensorflow/c

In [23]:
pos_test = os.path.join(POS_DIR, "test.txt")

test_sentences = format_data(pos_test, False)

dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)

In [27]:
pos_test_correct = os.path.join(POS_DIR, "test_labelled.txt") 
correct_test_sen = format_data(pos_test_correct)

In [28]:
def compare_results(predicted_sen, correct_sen, acc = True):
    total = 0
    correct_total = 0
    if acc:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    correct_total = correct_total + 1
                total = total + 1
        acc = correct_total / total
        return acc
    else:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    print(f"{predict[1]} \t {correct[1]}")
                else:
                    print(f"{predict[1]} \t {correct[1]} <------ Error")

In [31]:
test_sentences = format_data(pos_test, False)

preprocessed_test_data = preprocess_unlabelled_test_data(embeddings, oov, test_sentences)

In [33]:
predicted_data = []
arg_max_dict = []
def test_set_predictions(preprocessed_test_data, test_sentences):
    for sentence in preprocessed_test_data:
        predict_x=model.predict(sentence, batch_size=1, verbose=0) 
        predict_x = np.argmax(predict_x, axis=1)
        arg_max_dict.append(predict_x)
        
    for index in range(len(test_sentences)):
        predicted_sen = list(zip(test_sentences[index], le.inverse_transform(arg_max_dict[index])))
        predicted_data.append(predicted_sen)


In [34]:
test_set_predictions(preprocessed_test_data, test_sentences)

In [35]:
predicted_data[:2]

[[('rockwell', 'NNP'),
  ('international', 'NNP'),
  ('corp.', 'NNP'),
  ("'s", 'DT'),
  ('tulsa', 'NNP'),
  ('unit', 'VBP'),
  ('said', 'VBD'),
  ('it', 'PRP'),
  ('signed', 'VBD'),
  ('a', 'DT'),
  ('tentative', 'JJ'),
  ('agreement', 'NN'),
  ('extending', 'VBG'),
  ('its', 'PRP$'),
  ('contract', 'NN'),
  ('with', 'IN'),
  ('boeing', 'NNP'),
  ('co.', 'NNP'),
  ('to', 'TO'),
  ('provide', 'VB'),
  ('structural', 'JJ'),
  ('parts', 'NNS'),
  ('for', 'IN'),
  ('boeing', 'NNP'),
  ("'s", 'POS'),
  ('747', 'NN'),
  ('jetliners', 'NNS'),
  ('.', '.')],
 [('rockwell', 'NNP'),
  ('said', 'VBD'),
  ('the', 'DT'),
  ('agreement', 'NN'),
  ('calls', 'VBZ'),
  ('for', 'IN'),
  ('it', 'PRP'),
  ('to', 'TO'),
  ('supply', 'VB'),
  ('200', 'CD'),
  ('additional', 'JJ'),
  ('so-called', 'JJ'),
  ('shipsets', 'NNP'),
  ('for', 'IN'),
  ('the', 'DT'),
  ('planes', 'NNS'),
  ('.', '.')]]

In [38]:
compare_with_test_set(predicted_data, correct_test_sen)

95.89463241657344

In [133]:
def generate_labelled_data(file_name):
    f = open(file_name, "w")
    for sentence in predicted_data:
        for word, pos in sentence:
            f.write(f"{word} {pos}\n")
        f.write(f"\n")
    f.close()